In [4]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt


/home/mbergst/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
keypoints = {1: (44, 30), 2: (457, 30), 3: (44, 563), 4: (457, 563), 5: (44, 138), 6: (174, 138), 7: (174, 230), 8: (174, 346), 9: (174, 453), 10: (44, 453), 11: (44, 230), 12: (86, 230), 13: (86, 368), 14: (44, 368), 15: (130, 295), 16: (457, 230), 17: (457, 295), 18: (457, 368), 19: (201, 295), 20: (387, 295), 21: (530, 295), 22: (870, 30), 23: (870, 563), 24: (870, 138), 25: (740, 138), 26: (740, 230), 27: (740, 346), 28: (740, 453), 29: (870, 453), 30: (870, 230), 31: (827, 230), 32: (827, 368), 33: (870, 368), 34: (785, 295), 35: (712, 295)}

In [11]:
# Choose dataset 1 or 2:
dataset = 1  # 1 or 2

if dataset == 1:
    data_folder = "1_train-val_1min_aalesund_from_start"
    yaml_file = "yaml_files/keypoint_1.yaml"
    labels = "labels_1"

else:
    data_folder = "2_train-val_1min_after_goal"
    yaml_file = "yaml_files/keypoint_2.yaml"
    labels = "labels_2"

In [2]:
image_src_folder = f'/datasets/tdt4265/other/rbk/{data_folder}/img1'
label_src_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/{labels}'
image_dest_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/{data_folder}/images/'
label_dest_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/{data_folder}/labels/'

from scripts.keypoint_dataset import distribute_files

distribute_files(image_src_folder, label_src_folder, image_dest_folder, label_dest_folder)

In [22]:
# Last inn sist trente modell
# Gjør det ikke bedre virker det som

def find_latest_model_with_best(base_path, type='keypoints'):
    detect_path = os.path.join(base_path, 'runs', type)
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/mbergst/TDT4265_Project' 


#model = YOLO(find_latest_model_with_best(base_path, 'keypoints'))
#model = YOLO('yolov8n.pt')


In [ ]:
# Gjør det veldig bra på dataset 1
model = YOLO('/work/mbergst/TDT4265_Project/runs/keypoints/train54/weights/best.pt')

In [27]:
import yaml
from scripts.count_label_classes import count_label_classes
import os
from collections import Counter

def update_yaml(base_yaml_path, new_yaml_path, directory_path):
    # Teller klasser i label-filene
    class_counts = count_label_classes(directory_path)  # Antar funksjonen allerede er skrevet

    # Leser eksisterende YAML-fil
    with open(base_yaml_path, 'r') as file:
        yaml_config = yaml.safe_load(file)
    
    # Oppdaterer 'nc' og 'names' basert på faktiske klasser funnet
    existing_classes = sorted(class_counts.keys(), key=lambda x: int(x))  # Sorterer klassene numerisk
    yaml_config['nc'] = len(existing_classes)
    yaml_config['names'] = [str(x) for x in existing_classes]

    # Skriver den nye YAML-filen
    with open(new_yaml_path, 'w') as file:
        yaml.safe_dump(yaml_config, file, sort_keys=False)

# Kall funksjonen med riktig filsti og katalog
base_yaml_path = '/work/mbergst/TDT4265_Project/yaml_files/keypoint_2.yaml'
new_yaml_path = '/work/mbergst/TDT4265_Project/yaml_files/keypoint_test.yaml'
directory_path = '/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/labels_2'
update_yaml(base_yaml_path, new_yaml_path, directory_path)


Labels 1
 OrderedDict([(0, 1105), (1, 1494), (4, 960), (5, 1238), (6, 1167), (7, 902), (8, 569), (10, 820), (11, 901), (12, 543), (13, 161), (14, 895), (15, 1611), (16, 1623), (17, 1636), (18, 1161), (19, 721), (20, 2152), (21, 99), (24, 31), (25, 172), (26, 134), (38, 130)])
Labels 2
 OrderedDict([(1, 1802), (5, 614), (6, 302), (7, 2), (15, 1742), (17, 1758), (18, 274), (19, 1802), (20, 1802), (21, 423), (24, 304), (25, 724), (26, 504), (27, 287), (28, 134), (30, 232), (31, 274), (32, 121), (38, 499)])


In [15]:
results = model.train(data=yaml_file, epochs=8, batch=20, imgsz=(1600, 900), project='/work/mbergst/TDT4265_Project/runs/keypoints')

New https://pypi.org/project/ultralytics/8.2.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=/work/mbergst/TDT4265_Project/runs/keypoints/train69/weights/best.pt, data=yaml_files/keypoint_1.yaml, epochs=8, time=None, patience=100, batch=20, imgsz=(1600, 900), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/TDT4265_Project/runs/keypoints, name=train7232, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=F

train: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/labels/train.cache... 1441 images, 0 backgrounds, 150 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]

train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000340.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000342.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000343.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/train/000344.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
trai


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 37 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]

val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000341.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000351.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000366.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000376.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /wor

...

        8/8      18.9G      1.237      1.082      1.033        234       1600: 100%|██████████| 65/65 [00:16<00:00,  3.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  8.54it/s]

                   all        324       3668      0.974      0.973       0.98       0.63



8 epochs completed in 0.041 hours.
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/keypoints/train7232/weights/last.pt, 6.4MB
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/keypoints/train7232/weights/best.pt, 6.4MB

Validating /work/mbergst/TDT4265_Project/runs/keypoints/train7232/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3010133 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.68it/s]


                   all        324       3668      0.974      0.973       0.98       0.63
                     0        324        211      0.999      0.995      0.995      0.627
                     1        324        265       0.98          1      0.993      0.691
                     6        324        187      0.946      0.942      0.967       0.49
                     7        324        239          1      0.997      0.995      0.724
                     8        324        230      0.993          1      0.995      0.648
                    10        324        177      0.993      0.994      0.995      0.599
                    11        324        107          1      0.976       0.99      0.616
                    13        324        166      0.888      0.892      0.889      0.431
                    14        324        177      0.994      0.973      0.994      0.613
                    15        324        101      0.989       0.96      0.989       0.54
                    1

In [19]:
# Evaluér modellens prestasjon på valideringssettet
val_results = model.val()

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)


Model summary (fused): 168 layers, 3010133 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 37 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]

val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000341.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000351.jpg: ignoring corrupt image/label: Label class 25 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000366.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/1_train-val_1min_aalesund_from_start/images/val/000376.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 23. Possible class labels are 0-22
val: WARNING ⚠️ /wor


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.22it/s]


                   all        324       3668      0.974      0.973      0.979      0.632
                     0        324        211      0.999      0.995      0.995      0.626
                     1        324        265       0.98          1      0.993      0.694
                     6        324        187      0.941      0.936      0.958      0.487
                     7        324        239          1      0.997      0.995      0.724
                     8        324        230      0.993          1      0.995      0.655
                    10        324        177      0.994      0.994      0.995      0.599
                    11        324        107          1      0.976       0.99      0.615
                    13        324        166      0.894      0.892      0.891      0.436
                    14        324        177      0.994      0.973      0.994      0.615
                    15        324        101      0.989       0.96      0.989      0.551
                    1

In [23]:
# Tracking

all_frames = f'/datasets/tdt4265/other/rbk/{data_folder}/img1'
frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])


In [24]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_width=1):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene


image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000001.jpg: 928x1600 1 1, 1 18, 1 19, 1 20, 1 24, 1 25, 13.8ms
Speed: 3.3ms preprocess, 13.8ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000002.jpg: 928x1600 1 1, 1 18, 1 19, 1 20, 1 24, 1 25, 3.0ms
Speed: 3.4ms preprocess, 3.0ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000003.jpg: 928x1600 1 1, 1 18, 1 19, 1 20, 1 24, 1 25, 3.1ms
Speed: 3.8ms preprocess, 3.1ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /datasets/tdt4265/other/rbk/1_train-val_1min_aalesund_from_start/img1/000004.jpg: 928x1600 1 1, 1 18, 1 19, 1 20, 1 24, 1 25, 3.0ms
Speed: 3.4ms preprocess, 3.0ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /datasets/tdt4265/other/rbk